In [9]:
import pandas as pd

In [10]:
import os

In [11]:
diretorio = r'C:\Users\vladi\Downloads\quinta\matriculas'
arquivos = os.listdir(diretorio)

In [12]:
arquivos

['1555530593569.pdf', '8444428223126.pdf', '8787702651030.pdf']

In [13]:
from extrai_texto import extract_text_from_pdf as extrair_texto

In [16]:
texto = []
for arquivo in arquivos:
    matricula = {
        'local_arquivo': diretorio + '\\' + arquivo,
        'conteudo': None
    }
    
    try:
        # Usando a função com parâmetros otimizados
        conteudo = extrair_texto(
            matricula['local_arquivo'],
            min_text_length=1000,  # Ajuste conforme necessário
            apply_ocr_if_less=True
        )
        
        if conteudo:
            matricula['conteudo'] = conteudo
            texto.append(matricula)
        else:
            print(f"Atenção: Nenhum conteúdo extraído de {arquivo}")
            
    except Exception as e:
        print(f"Erro ao processar {arquivo}: {str(e)}")
        continue

# Verificação dos resultados
for i, item in enumerate(texto):
    print(f"\nArquivo {i+1}: {item['local_arquivo']}")
    print(f"Tamanho do texto: {len(item['conteudo']) if item['conteudo'] else 0} caracteres")
    

Texto insuficiente (84 caracteres). Aplicando OCR...

Arquivo 1: C:\Users\vladi\Downloads\quinta\matriculas\1555530593569.pdf
Tamanho do texto: 3393 caracteres

Arquivo 2: C:\Users\vladi\Downloads\quinta\matriculas\8444428223126.pdf
Tamanho do texto: 2899 caracteres

Arquivo 3: C:\Users\vladi\Downloads\quinta\matriculas\8787702651030.pdf
Tamanho do texto: 17027 caracteres


In [17]:
texto

[{'local_arquivo': 'C:\\Users\\vladi\\Downloads\\quinta\\matriculas\\1555530593569.pdf',
  'conteudo': 'Páigina 1 / 5\n8º REGISTRO DE IMÓVEIS DE CURITIBA \nCódigo Nacional de Matrícula: 83238.2.0178864-36 \nCERTIDÃO\nAssinado por: CARLA RUBIA DOS \n \nSANTOS\nNo dia: 30/07/2024\nDocumento assinado eletronicamente com \ncertificado digitalexpedido nos parâmetros da \nICP-Brasil, nos termos da Medida Provisória nº \n2.200-2, de 24/08/2001.\nConsulte a autenticidade desta certidão em: https://8ricuritiba-pr.sistemaasgard.com.br/#/validar-assinatura, utilizando o código de verificação 9XRZ-WRSB-D3XL-Z8DY.\n\nPáigina 2 / 5\n8º REGISTRO DE IMÓVEIS DE CURITIBA \nCódigo Nacional de Matrícula: 83238.2.0178864-36 \nCERTIDÃO\nAssinado por: CARLA RUBIA DOS \n \nSANTOS\nNo dia: 30/07/2024\nDocumento assinado eletronicamente com \ncertificado digitalexpedido nos parâmetros da \nICP-Brasil, nos termos da Medida Provisória nº \n2.200-2, de 24/08/2001.\nConsulte a autenticidade desta certidão em: https

In [18]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import io
import re

def extract_max_text(path, min_text_length=100):
    """
    Extrai o máximo de texto possível combinando métodos direto e OCR,
    com verificação por página para otimização.
    
    Retorna:
        dict: {
            'texto_completo': texto combinado,
            'paginas_validas': nº de páginas com texto válido,
            'metodos_por_pagina': lista dos métodos usados por página
        }
    """
    # Configuração do Tesseract (ajuste para seu Windows)
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    result = {
        'texto_completo': '',
        'paginas_validas': 0,
        'metodos_por_pagina': []
    }
    
    # Método 1: Extração direta com PyMuPDF
    texto_direto_por_pagina = []
    with fitz.open(path) as doc:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            texto = page.get_text().strip()
            texto_direto_por_pagina.append(texto)
            
            # Verifica se o texto é válido
            if len(texto) >= min_text_length:
                result['metodos_por_pagina'].append('DIRETO')
                result['paginas_validas'] += 1
            else:
                result['metodos_por_pagina'].append('OCR NECESSÁRIO')
    
    # Se todas as páginas têm texto válido, retorna
    if all(m == 'DIRETO' for m in result['metodos_por_pagina']):
        result['texto_completo'] = '\n\n'.join(texto_direto_por_pagina)
        return result
    
    # Método 2: OCR para páginas problemáticas
    texto_combinado = []
    with fitz.open(path) as doc:
        for page_num in range(len(doc)):
            # Reutiliza texto direto se for válido
            if result['metodos_por_pagina'][page_num] == 'DIRETO':
                texto_combinado.append(texto_direto_por_pagina[page_num])
                continue
                
            # Tenta OCR para a página
            try:
                # Primeiro tenta extrair de imagens embutidas
                page = doc.load_page(page_num)
                texto_ocr = ''
                
                # Extrai imagens da página
                for img_index, img in enumerate(page.get_images(full=True)):
                    xref = img[0]
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    image = Image.open(io.BytesIO(image_bytes))
                    
                    # Configurações otimizadas para documentos brasileiros
                    custom_config = r'--oem 3 --psm 6 -l por+eng'
                    texto_ocr += pytesseract.image_to_string(image, config=custom_config) + '\n'
                
                # Se OCR de imagens embutidas não foi suficiente, converte a página completa
                if len(texto_ocr.strip()) < min_text_length:
                    images = convert_from_path(path, first_page=page_num+1, last_page=page_num+1)
                    if images:
                        texto_ocr = pytesseract.image_to_string(images[0], config=custom_config)
                
                if texto_ocr.strip():
                    result['metodos_por_pagina'][page_num] = 'OCR'
                    result['paginas_validas'] += 1
                    texto_combinado.append(texto_ocr)
                else:
                    result['metodos_por_pagina'][page_num] = 'FALHA'
                    
            except Exception as e:
                print(f"Erro no OCR da página {page_num+1}: {str(e)}")
                result['metodos_por_pagina'][page_num] = 'FALHA'
                continue
    
    result['texto_completo'] = '\n\n'.join(texto_combinado)
    return result

def extrair_dados_imovel(texto_completo):
    """
    Extrai dados específicos do imóvel do texto combinado
    """
    dados = {
        'matricula': None,
        'endereco': None,
        'proprietario': None
    }
    
    # Expressões regulares para encontrar os dados
    padrao_matricula = re.compile(r'Mat[ríí]cula\s*N?[º°]?\s*([\d.,-]+)', re.IGNORECASE)
    padrao_codigo_nacional = re.compile(r'C[óo]digo\s*Nacional\s*de\s*Mat[ríí]cula:\s*([\d.-]+)')
    
    # Tenta encontrar a matrícula
    for padrao in [padrao_codigo_nacional, padrao_matricula]:
        match = padrao.search(texto_completo)
        if match:
            dados['matricula'] = match.group(1).strip()
            break
    
    # Extrai endereço (procura por padrões comuns)
    padrao_endereco = re.compile(
        r'(?:IM[ÓO]VEL|Localiza..o|Endere.o)[:\s]*(.*?)(?:\n|$)|'
        r'(Rua|Avenida|Av\.|Praça|Travessa)\s+([^,\n]+),\s*(n?[º°]?\s*\d+[^\n]*)',
        re.IGNORECASE | re.DOTALL
    )
    
    match = padrao_endereco.search(texto_completo)
    if match:
        dados['endereco'] = ' '.join([g for g in match.groups() if g]).strip()
    
    # Extrai proprietário (procura por termos jurídicos)
    padrao_proprietario = re.compile(
        r'(PROPRIET[AÁ]RIO|TITULAR|ADQUIRENTE)[:\s]*(.*?)(?:\n|$)'
        r'|(?:Consolidada a propriedade em nome de|vendido para|transfere para)\s*(.*?)(?:\n|$)',
        re.IGNORECASE | re.DOTALL
    )
    
    match = padrao_proprietario.search(texto_completo)
    if match:
        dados['proprietario'] = ' '.join([g for g in match.groups() if g]).strip()
    
    return dados

# Exemplo de uso:
arquivos = [
    'C:\\Users\\vladi\\Downloads\\quinta\\matriculas\\1555530593569.pdf',
    'C:\\Users\\vladi\\Downloads\\quinta\\matriculas\\8444428223126.pdf',
    'C:\\Users\\vladi\\Downloads\\quinta\\matriculas\\8787702651030.pdf'
]

resultados = []
for arquivo in arquivos:
    print(f"\nProcessando: {arquivo}")
    
    # Extrai o máximo de texto possível
    extracao = extract_max_text(arquivo)
    
    # Extrai dados específicos
    dados = extrair_dados_imovel(extracao['texto_completo'])
    
    # Armazena resultados
    resultados.append({
        'arquivo': arquivo,
        'metodos_por_pagina': extracao['metodos_por_pagina'],
        'dados': dados,
        'texto_completo': extracao['texto_completo'][:500] + "..."  # Amostra do texto
    })
    
    print(f"Métodos usados por página: {extracao['metodos_por_pagina']}")
    print(f"Dados extraídos: {dados}")

# resultados contém todos os dados coletados


Processando: C:\Users\vladi\Downloads\quinta\matriculas\1555530593569.pdf
Métodos usados por página: ['DIRETO', 'DIRETO', 'DIRETO', 'DIRETO', 'DIRETO']
Dados extraídos: {'matricula': None, 'endereco': None, 'proprietario': None}

Processando: C:\Users\vladi\Downloads\quinta\matriculas\8444428223126.pdf
Métodos usados por página: ['DIRETO', 'DIRETO', 'DIRETO', 'DIRETO']
Dados extraídos: {'matricula': None, 'endereco': None, 'proprietario': None}

Processando: C:\Users\vladi\Downloads\quinta\matriculas\8787702651030.pdf
Métodos usados por página: ['OCR', 'OCR', 'OCR']
Dados extraídos: {'matricula': None, 'endereco': 'Fração ideal do solo e partes comuns de 0,004831088, que co wspondera ao', 'proprietario': 'proprietário ou possuidor do terreno'}
